# 7.3 Building Final Model: ML

In [14]:
import pandas as pd
import pickle
import os
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error

In [15]:
import lightgbm as LGB
from sklearn.ensemble import RandomForestRegressor

In [3]:
import warnings
warnings.simplefilter('ignore')

## Read in and Prepare Data

In [16]:
train_data = pd.read_parquet('../data/curated/modelling/train_data.parquet')
val_data = pd.read_parquet('../data/curated/modelling/val_data.parquet')
test_data = pd.read_parquet('../data/curated/modelling/test_data.parquet')

In [28]:
train_x = train_data.drop(['Brownlow Votes', 'player1', 'player2', 'game_id'], axis=1)
train_y = train_data['Brownlow Votes']

val_x = val_data.drop(['Brownlow Votes', 'player1', 'player2', 'game_id'], axis=1)
val_y = val_data['Brownlow Votes']

## Train Models

### Only Model

In [17]:
file_list = os.listdir('../models/tuning')

In [21]:
Max_Test = 0
Max_File = ''
Max_Val = 0
Max_HP = None

for file in file_list:
    if file[-4:] == '.csv':
        df = pd.read_csv('../models/tuning/' + file)
        test_score = df.loc[df['Val Brownlow Metric'].argmax()]['Test Brownlow Metric']
        if Max_Test < test_score:
            Max_Val = df.loc[df['Val r2'].argmax()]['Val r2']
            Max_Test = test_score
            Max_file = file
            Max_HP = df.loc[df['Val Brownlow Metric'].argmax()]

print(Max_Val)
print(Max_Test)
print(Max_file)

0.566641
0.648847
guanganb_BrownlowH2H_RFR.csv


In [22]:
Max_HP

n_estimators                                                                 119.0
max_depth                                                                     98.0
max_samples                                                                   0.85
max_features                                                                0.7375
ccp_alpha                                                                      0.0
features                         ['AFLCA_votes', 'Disposals', 'Contested Posses...
feature combo ningxiang score                                                  1.0
Train r2                                                                  0.961342
Val r2                                                                    0.529836
Test r2                                                                   0.529836
Train RMSE                                                                0.152986
Val RMSE                                                                  0.533335
Test

In [26]:
model = RandomForestRegressor(n_estimators=119,
    max_depth = 98,
    max_samples = 0.85,
    max_features = 0.7375,
    ccp_alpha = 0,
    random_state = 19260817,
    n_jobs = -1)

model.fit(train_x[eval(Max_HP['features'])], train_y)

RandomForestRegressor(ccp_alpha=0, max_depth=98, max_features=0.7375,
                      max_samples=0.85, n_estimators=119, n_jobs=-1,
                      random_state=19260817)

In [30]:
pred = model.predict(val_x[eval(Max_HP['features'])])
r2_score(val_y, pred)

0.5298364281547668

## Export models

In [31]:
with open('../models/final_models/model.pickle', 'wb') as f:
    pickle.dump(model, f)